In [ ]:
#To connect with google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Installings
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer


In [ ]:
import csv
import random
import datasets
import pandas as pd
#Function to genrate dataset accordingly
def getDataSet(p):
  ds=pd.read_csv(p,sep=',',quoting=csv.QUOTE_NONE,encoding='utf-8')
  nv=float("NAN")
  ds.replace("",nv,inplace=True)
  ds.dropna(subset=["sentence"],inplace=True)
  return ds

In [ ]:
#Getting datasets
path_train="/content/drive/My Drive/NLP/DataSet/Training_file.csv"
path_test="/content/drive/My Drive/NLP/Test-DataSet/test_file.csv"


dataframe_train=getDataSet(path_train)
dataframe_test=getDataSet(path_test)

#Creating Datasets
training_ds=datasets.Dataset.from_pandas(dataframe_train)
test_ds=datasets.Dataset.from_pandas(dataframe_test)


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
#Function to generate the
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(training_ds.remove_columns(["path"]), num_examples=20)


,sentence,__index_level_0__
0,ڈاون کیوں کرنا چاہ رہے ہیں سر,360
1,بس آپ بے فکر ہو جائیں میں ھل کرواتا ہوں,389
2,55,168
3,پی ٹی آئی کا معنی,399
4,تھینک یوویلکم سر,282
5,وہ نیکسٹ منتھ سے کٹ جائے گا,244
6,دس کا لوڈ کیا جاتا ہے لیکن,119
7,حاجی ڈبل بن ڈبل ٹو,70
8,نومبر تک مطلب کے,295
9,حادثے کے لیے معذرت چاہتا ہوں,249


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
#Function to remove Special Charaters
def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [ ]:
#Removing Special Characters
training_ds = training_ds.map(remove_special_characters)
test_ds = test_ds.map(remove_special_characters)

In [ ]:
#Showing the random rows from training dataset after removing the special characters
show_random_elements(training_ds.remove_columns(["path"]))

,__index_level_0__,sentence
0,130,4 ہے 345 ہے
1,408,1035 ہے
2,364,صحیح ٹھیک ہے
3,366,جی جی
4,383,اچھا جو ایسے آ رہا ہے
5,326,چار سو روپے سر
6,117,دو سو روپے پڑھیں گے
7,82,اور یہ میں آپ کوشور آڈی دلوا دو گے مسئلہ کوئی
8,470,پی ٹی سی ایل ڈاٹ نیٹ ڈاٹ پی کے
9,80,ابھی چیک کریں گے تو ایڈ کوئی آتا ہی نہیں


In [ ]:
#Function to exract chars from text used in text
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
#Extracting characters
vocab_train = training_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_ds.column_names)
vocab_test = test_ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_ds.column_names)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
#Showing vocab_dict
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 11,
 '0': 63,
 '1': 37,
 '2': 46,
 '3': 35,
 '4': 38,
 '5': 49,
 '6': 42,
 '7': 1,
 '8': 21,
 '9': 10,
 'c': 39,
 'k': 8,
 'l': 64,
 'm': 57,
 'o': 25,
 'p': 50,
 't': 23,
 'ء': 4,
 'آ': 41,
 'ؤ': 28,
 'ئ': 26,
 'ا': 53,
 'ب': 34,
 'ت': 31,
 'ث': 27,
 'ج': 7,
 'ح': 2,
 'خ': 15,
 'د': 65,
 'ذ': 58,
 'ر': 5,
 'ز': 54,
 'س': 61,
 'ش': 40,
 'ص': 45,
 'ض': 55,
 'ط': 47,
 'ظ': 56,
 'ع': 24,
 'غ': 0,
 'ف': 14,
 'ق': 33,
 'ك': 43,
 'ل': 17,
 'م': 60,
 'ن': 20,
 'و': 6,
 'َ': 12,
 'ُ': 52,
 'ِ': 3,
 'ْ': 48,
 'ٹ': 44,
 'پ': 22,
 'چ': 13,
 'ڈ': 18,
 'ڑ': 36,
 'ک': 16,
 'گ': 30,
 'ں': 59,
 'ھ': 32,
 'ہ': 9,
 'ۃ': 19,
 'ی': 29,
 'ے': 62,
 '\u200f': 51}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
#Showing the len of the vocab_dict
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

68

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
#Showing the third row of training dataset
training_ds[3]

{'__index_level_0__': 4,
 'path': '/content/drive/MyDrive/NLP/DataSet/Audio Files/9320619f-4123-444b-bcf4-18a559496595.wav',
 'sentence': 'چارجز آپ کے ایکسٹرا ہی چارج ہوتے ہیں '}

In [ ]:
import torchaudio

#Function to convert speechfile to array_fn
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
training_ds = training_ds.map(speech_file_to_array_fn, remove_columns=training_ds.column_names)
test_ds = test_ds.map(speech_file_to_array_fn, remove_columns=test_ds.column_names)

In [ ]:
#Playing Random
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(training_ds)-1)

ipd.Audio(data=np.asarray(training_ds[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
#Just getting random target text
rand_int = random.randint(0, len(training_ds)-1)

print("Target text:", training_ds[rand_int]["target_text"])
print("Input array shape:", np.asarray(training_ds[rand_int]["speech"]).shape)
print("Sampling rate:", training_ds[rand_int]["sampling_rate"])

Target text: میں یہاں سے ایک دفعہ ریفریش 
Input array shape: (34400,)
Sampling rate: 16000


In [ ]:
#Function to prepare dataset
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
#Preparing dataset
training_ds = training_ds.map(prepare_dataset, remove_columns=training_ds.column_names, batch_size=8, num_proc=4, batched=True)
test_ds = test_ds.map(prepare_dataset, remove_columns=test_ds.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
#Collecting data
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/Urdu_demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

'from transformers import TrainingArguments\n\ntraining_args = TrainingArguments(\n  output_dir="/content/gdrive/MyDrive/Urdu_demo",\n  # output_dir="./wav2vec2-large-xlsr-turkish-demo",\n  group_by_length=True,\n  per_device_train_batch_size=16,\n  gradient_accumulation_steps=2,\n  evaluation_strategy="steps",\n  num_train_epochs=30,\n  fp16=True,\n  save_steps=400,\n  eval_steps=400,\n  logging_steps=400,\n  learning_rate=3e-4,\n  warmup_steps=500,\n  save_total_limit=2,\n)'

In [ ]:
trainer.train()

Trained Successfully


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/urdu-demo").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/urdu-demo")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained


In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug


In [ ]:
print("Prediction:")
print(prediction[0]["sentence"])

print("\nReference:")
print(test_transcription[0]["sentence"].lower())

Prediction:
اسلام علیکم یاد کر رہا ہں جی فرمائے آپ سے کیا مدد کر سکتا ہوں

Reference:
اسلام علیکم بات کر رہا ہوں جی فرمائے میں آپکی کیا مدد کر سکتا ہوں


In [ ]:

#################Module to Calculate Edit Distance between Reference and Prediction##################
##################################Implemented By Self#######################################
Edit_Distance=0
#Function to genrate array of words
def wordTokenizer(text):
    words=[]
    current_word=""
    for c in text:
      #Detecting word boundries
      if c==" "  or c=="۔" or ord(c)==10 or ord(c)==9:
          if current_word!="":
              words.append(current_word)
              current_word=""
      else:
        current_word=current_word+c

    if current_word!="":
      words.append(current_word)

    return words



#Function to count word occurance in sequence
def getWordCount(word,sequence):
  #sequence=wordTokenizer(sequence)
  count=0
  for w in sequence:
    if w==word:
      count=count+1

  return count


s=769
#Function to get optimal mapping from words to word
def getMapping(ref,pre):
  ref=wordTokenizer(ref)
  pre=wordTokenizer(pre)

  mapped=[]
  map={}
  n=0
  #Loop for each char in src
  for j in range(0,len(ref)):
    s=ref[j]
    for i in range(n,len(pre)):
      t=pre[i]
      #If can be mapped
      if s==t:
        mapped.append(s)
        map[j]=i
        n=i+1
        break
      else:

          ref_count_c = getWordCount(pre[i], ref[j:len(ref)])
          pre_count_c=getWordCount(pre[i],pre[i:len(pre)])
          #Check if we cross a char or not to get optimal mapping
          if ref_count_c<=pre_count_c and ref_count_c>0:
             #print(tar[i]+str(i)+" Cant sacrifise to map "+s)
             n = i
             break


  #print(mapped)
  return map




total_sentences=s
#Function to detect missed maped Words
def MissedMaps(ref,pre,mapping):
    ref=wordTokenizer(ref)
    pre=wordTokenizer(pre)

    s_index = 0
    t_index = 0
    current_key=-1
    ref_misseds=[]
    pre_misseds=[]
    #Loop for each mapped char
    for key in mapping:
        current_key=key
        s_i = key;
        t_i = mapping[s_i]
        ref_sub = ""
        pre_sub = ""
        #Loop to concatinate ref miss mapped words
        for i in range(s_index, s_i):
          ref_sub=ref_sub+ref[i]+" "
        for i in range(t_index, t_i):
          pre_sub=pre_sub+pre[i]+" "

        #Updatings
        if not (ref_sub=="" and pre_sub==""):
          ref_misseds.append(ref_sub)
          pre_misseds.append(pre_sub)
        s_index=s_i+1
        t_index=t_i+1
    #Getting words from last
    ref_sub = ""
    pre_sub = ""
    for i in range(s_index, len(ref)):
        ref_sub=ref_sub+ref[i]+" "
    for i in range(t_index,len(pre)):
        pre_sub=pre_sub+pre[i]+" "

    #Updatings
    if not (ref_sub=="" and pre_sub==""):
      ref_misseds.append(ref_sub)
      pre_misseds.append(pre_sub)

    l=[ref_misseds,pre_misseds]
    return l




#Function to detect Operations
def detectOprs(src,tar):
    global Edit_Distance
    src=wordTokenizer(src)
    tar=wordTokenizer(tar)

    src_list=[]
    tar_list=[]
    #Genrating two equal lists to map missed mapping
    if len(src)>len(tar):
        for i in range(0,len(src)):
            src_list.append(src[i])
            try:
                tar_list.append(tar[i])
            except:
                tar_list.append("")

    else:
        for i in range(0,len(tar)):
            tar_list.append(tar[i])
            try:
                src_list.append(src[i])
            except:
                src_list.append("")

    Edit_Distance=Edit_Distance+len(src_list)




#Function to calculate the Word Error Rate
def getWordErrorRate(ref,pre):
     global Edit_Distance
     mapping=getMapping(ref,pre)
     l=MissedMaps(ref,pre,mapping)
     ref_subs=l[0]
     pre_subs=l[1]
     for i in range(0,len(ref_subs)):
        detectOprs(ref_subs[i],pre_subs[i])

     t=len(wordTokenizer(ref))
     wer=Edit_Distance/(t)
     Edit_Distance=0
     return wer









wer=0
for i in range(0,169):
  pre=prediction[i]["sentence"]
  ref=test_transcription[i]["sentence"]
  wer=wer+getWordErrorRateOverTestDataSet(pre,ref)


wer=wer/total_sentences
print("Word Error Rate Over the Whole Test Set:"+str(wer))





Word Error Rate Over the Whole Test Set:0.27837017771998374


# **Research Report:**
## *Introduction*:
Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in September 2020 by Alexei Baevski, Michael Auli, and Alex Conneau. Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, Facebook AI presented XLSR-Wav2Vec2 (click here). XLSR stands for cross-lingual speech representations and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to BERT's masked language modeling, the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.
## *Prepare Data, Tokenizer, Feature Extractor*:
ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, e.g. a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called Wav2Vec2CTCTokenizer, and a feature extractor, called Wav2Vec2FeatureExtractor.

## *Create Wav2Vec2CTCTokenizer*:
The pretrained Wav2Vec2 checkpoint maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, e.g., after pretraining a linear layer is added on top of BERT's embeddings for further classification - cf. with *"BERT"* section of this blog post.

The output size of this layer corresponds to the number of tokens in the vocabulary, which does not depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

## *Create Wav2Vec2CTCTokenizer*:
We can see that the transcriptions contain some special characters, such as ,.?!;:. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. E.g., the letter "s" has a more or less clear sound, whereas the special character "." does not. Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

## *Create XLSR-Wav2Vec2 Feature Extractor*:
Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called sampling. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the real speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, e.g., doubling the sampling rate results in data points being twice as long. Thus, before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of Babel, Multilingual LibriSpeech (MLS), and Common Voice. Most of those datasets were sampled at 16kHz, so that Common Voice, sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following.

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.


## *End of Documentation*:


